# First Sneaker Monitor (adidas waiting rooms)

In [ ]:
import time
import json
import requests
from discord_webhook import DiscordWebhook

#Input region -> SG/MY/AU/BR etc to change url destination
region = (input("Region: ")).upper()
url = "https://www.adidas.com/hpl/content/yeezy/config/" + str(region) + "/waitingRoomConfig.json"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36','Accept-Encoding': 'gzip, deflate, br','Accept-Language': 'en-US,en;q=0.9,hi;q=0.8'}
dc_url = "https://discordapp.com/api/webhooks/748631402479616111/Y9yUCRecKP5PrN8vywbHPcFnwRWTZUlE2vwFSrc8uE5FzCkzyzVXOGiMBdh0X8Xtxi9f"

#BOT username
botname = str(region) + " Waiting Room BOT"

#finding country flag for title
if region == "SG":
    flag = ":flag_sg:"
elif region == "MY":
    flag = ":flag_my:"
elif region == "AU":
    flag = ":flag_au:"
elif region == "GB":
    flag = ":flag_gb:"

#start timing
start_time = time.time()
duration = int(99999)
run_count = 1
count = 0

#check_duration in seconds is the frequency you wanna see the updates e.g. 300s = 5 minutes per update
check_duration = 300

#First scrape to set basis of comparison for subsequent scrapes
main_json = requests.get(url, headers = headers)

#json string converted to dictionary -> monitor changes in values of "statusMessageKey", "enableRecaptchaV3" and "productIds"
json_first = json.loads(main_json.text)

#First status, captcha, SKU info to update later
status = json_first['statusMessageKey']
captcha = json_first['enableRecaptchaV3']
SKU = json_first['productIds']

while True:
    #Main block to execute
    try:
        current_time = time.time()
        elapsed_time = current_time - start_time

        if elapsed_time < duration:
            if elapsed_time < (run_count * check_duration):
                main_json = requests.get(url, headers = headers)
                json_dict = json.loads(main_json.text)

                #Where discord webhook info will go
                data = {}
                data["embeds"] = []
                embed = {}

                #This portion is only gone through once, afterwhich the program will just keep looking for changes
                #if the scraped API data is exactly the same -> if the room is in welcome
                if json_dict == json_first:
                    #If the code is run when the status is 'welcome' / 'sale_started' / 'sale_over' -> webhook sent accordingly but just once
                    if status == 'welcome':
                        if count == 0:
                            data["username"] = botname
                            embed["title"] = "adidas  " + flag + "  Waiting Room Check Run - Sale not yet started"
                            embed["description"] = "Status: " + str(status) + "\nRecaptcha: " + str(captcha) + "\nProduct IDs: " + str(SKU)
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})
                            #Add count here to prevent repeatedly webhooking this
                            count += 1

                    elif status == 'sale_started':
                        if count == 0:
                            data["username"] = botname
                            embed["title"] = "adidas  " + flag + "  Waiting Room Check Run - Sale in progress"
                            embed["description"] = "Status: " + str(status) + "\nRecaptcha: " + str(captcha) + "\nProduct IDs: " + str(SKU)
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})
                            #Add count here to prevent repeatedly webhooking this
                            count += 1

                    elif status == 'sale_over':
                        if count == 0:
                            data["username"] = botname
                            embed["title"] = "adidas  " + flag + "  Waiting Room Check Run - Sale just over"
                            embed["description"] = "Status: " + str(status) + "\nRecaptcha: " + str(captcha) + "\nProduct IDs: " + str(SKU)
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})
                            #Add count here to prevent repeatedly webhooking this
                            count += 1

                #This portion will be run multiple times -> just keep on detecting when there are changes to the API data
                #If the scraped API data is different -> if sta_new is 'welcome' / 'sale_started' -> webhook updates sent accordingly
                else:
                    sta_new = json_dict['statusMessageKey']
                    cap_new = json_dict['enableRecaptchaV3']
                    SKU_new = json_dict['productIds']

                    #If the previous scraped API status was 'welcome', this portion will be executed
                    if status == 'welcome':
                        #'welcome' to 'welcome' but changes detected in the API
                        if (sta_new == 'welcome'):
                            data["username"] = botname
                            embed["title"] = "CHANGES DETECTED in adidas  " + flag
                            embed["description"] = "Status: " +  str(sta_new) + "\nRecaptcha: " + str(cap_new) + "\nProduct IDs: " + str(SKU_new)
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #Take over the latest JSON dict, keys, values
                            json_first = json_dict
                            status = sta_new
                            captcha = cap_new
                            SKU = SKU_new

                        #'welcome' to 'sale_started'
                        elif (sta_new == "sale_started"):
                            data["username"] = botname
                            data["content"] = "@here"
                            embed["title"] = "SALE STARTED on adidas  " + flag
                            embed["description"] = "Status: " + str(sta_new) + "\nRecaptcha: " + str(cap_new) + '\nProduct IDs: ' + str(SKU_new)
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #Take over the latest JSON dict, keys, values
                            json_first = json_dict
                            status = sta_new
                            captcha = cap_new
                            SKU = SKU_new

                        #'welcome' to 'sale_over'
                        elif (sta_new == "sale_over"):
                            data["username"] = botname
                            embed["title"] = "SALE OVER on adidas  " + flag
                            embed["description"] = "Status: " + str(sta_new) + "\nRecaptcha: " + str(cap_new) + '\nProduct IDs: ' + str(SKU_new)
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #Take over the latest JSON dict, keys, values
                            json_first = json_dict
                            status = sta_new
                            captcha = cap_new
                            SKU = SKU_new

                    #If the previous scraped API status was 'sale_started', this portion will be executed
                    elif status == 'sale_started':
                        #'sale_started' to 'welcome'
                        if (sta_new == 'welcome'):
                            data["username"] = botname
                            embed["title"] = "adidas  " + flag + "  status back to WELCOME"
                            embed["description"] = "Status: " +  str(sta_new) + "\nRecaptcha: " + str(cap_new) + "\nProduct IDs: " + str(SKU_new)
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #Take over the latest JSON dict, keys, values
                            json_first = json_dict
                            status = sta_new
                            captcha = cap_new
                            SKU = SKU_new

                        #'sale_started' remains as 'sale_started'
                        elif (sta_new == "sale_started"):
                            data["username"] = botname
                            embed["title"] = "SALE STARTED on adidas  " + flag
                            embed["description"] = "Status: " + str(sta_new) + "\nRecaptcha: " + str(cap_new) + '\nProduct IDs: ' + str(SKU_new)
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #Take over the latest JSON dict, keys, values
                            json_first = json_dict
                            status = sta_new
                            captcha = cap_new
                            SKU = SKU_new

                        #'sale_started' to 'sale_over'
                        elif (sta_new == "sale_over"):
                            data["username"] = botname
                            embed["title"] = "SALE OVER on adidas  " + flag
                            embed["description"] = "Status: " + str(sta_new) + "\nRecaptcha: " + str(cap_new) + '\nProduct IDs: ' + str(SKU_new)
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #Take over the latest JSON dict, keys, values
                            json_first = json_dict
                            status = sta_new
                            captcha = cap_new
                            SKU = SKU_new

                    #If the previous scraped API status was 'sale_over', this portion will be executed
                    elif status == 'sale_over':
                        #'sale_over' to 'welcome'
                        if (sta_new == 'welcome'):
                            data["username"] = botname
                            embed["title"] = "adidas  " + flag + "  status back to WELCOME"
                            embed["description"] = "Status: " +  str(sta_new) + "\nRecaptcha: " + str(cap_new) + "\nProduct IDs: " + str(SKU_new)
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #Take over the latest JSON dict, keys, values
                            json_first = json_dict
                            status = sta_new
                            captcha = cap_new
                            SKU = SKU_new

                        #'sale_over' to 'sale_started'
                        elif (sta_new == "sale_started"):
                            data["username"] = botname
                            embed["title"] = "SALE STARTED on adidas  " + flag
                            embed["description"] = "Status: " + str(sta_new) + "\nRecaptcha: " + str(cap_new) + '\nProduct IDs: ' + str(SKU_new)
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #Take over the latest JSON dict, keys, values
                            json_first = json_dict
                            status = sta_new
                            captcha = cap_new
                            SKU = SKU_new

                        #'sale_over' to 'sale_started'
                        elif (sta_new == "sale_started"):
                            data["username"] = botname
                            embed["title"] = "SALE STARTED on adidas  " + flag
                            embed["description"] = "Status: " + str(sta_new) + "\nRecaptcha: " + str(cap_new) + '\nProduct IDs: ' + str(SKU_new)
                            data["embeds"].append(embed)

                            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})

                            #Take over the latest JSON dict, keys, values
                            json_first = json_dict
                            status = sta_new
                            captcha = cap_new
                            SKU = SKU_new

            #This portion will update on command prompt to inform user that the program is running and not crashed -> vary the check_duration (seconds) to change the duration of updates
            elif elapsed_time >= (run_count * check_duration):
                cur_time = int(time.time())
                left = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(cur_time))

                right = "Program is running"
                print(str(left) + ": " + right)

                #Reset the start_time to make the loop run infinitely
                if run_count == 10:
                    run_count = 0
                    start_time = time.time()

                run_count += 1

        #if any errors, this webhook will be sent
        else:
            data["username"] = botname
            embed["title"] = "An error occured - the program is not running anymore"
            data["embeds"].append(embed)

            requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})
            break

    #Handling ConnectionError, TimeoutError where it will print out the error and keep waiting for 1 minute till making the next request
    except (ConnectionError, TimeoutError) as error:
        print(error)
        print("Will retry in 1 minute")
        time.sleep(60)

    #KeyboardInterrupt (ctrl C) will stop program and send another webhook to say the program has been stopped
    except KeyboardInterrupt:
        data = {}
        data["embeds"] = []
        embed = {}

        data["username"] = botname
        embed["title"] = "The program has been shut down"
        data["embeds"].append(embed)

        requests.post(dc_url, data=json.dumps(data), headers={"Content-Type": "application/json"})
        break


### Usable when in adidas waiting rooms. When people are in the waiting room, the exact model of sneaker or time which the sneakers get released are not obvious. However, using knowledge of the waiting room API, the program is capable of detecting backend changes. With the various status changes, the user is able to know the exact release time to use their multi-purchase bot to maximize their opportunity of success. 

In [12]:
%%html
<img src="img/2.jpg">

### Linked to discord channel to notify users via webhooks. Also, there are several lines of code added to enable the code to run continuously 24/7

In [13]:
%%html
<img src="img/3.png">